In [1]:
import struct
import binascii

In [2]:
s = struct.Struct('I 2s f')
values = (1, 'ab', 2.7)
print 'Original:', values

Original: (1, 'ab', 2.7)


In [3]:
import array
a = array.array('c', '\0' * s.size)
print 'Before  :', binascii.hexlify(a)
s.pack_into(a, 0, *values)
print 'After   :', binascii.hexlify(a)
print 'Unpacked:', s.unpack_from(a, 0)

Before  : 000000000000000000000000
After   : 0100000061620000cdcc2c40
Unpacked: (1, 'ab', 2.700000047683716)


In [68]:
class Parcel(object):
    def __init__(self):
        self.block = block = 50
        self._data_pointer = 0
        self._data_size = 0
        self._buffer = array.array('c')
        self.setDataCapacity(block)
        primitives = [('byte', 'h'),
                      ('double', 'd'),
                      ('float', 'f'),
                      ('integer', 'i'),
                      ('long', 'q'),
                      ('string', 'p'),
                     ]
        self.primitives = dict((x[0], struct.Struct(x[1])) for x in primitives)
        pass
    
    def dataAvail(self):
        """
        Returns the amount of data remaining to be read from the
        parcel.
        """
        return self.dataSize() - self.dataPosition()

    def dataCapacity(self):
        """
        Returns the total amount of space in the parcel.
        """
        return len(self._buffer)

    def dataPosition(self):
        """
        Returns the current position in the parcel data.
        """
        return self._data_pointer

    def dataSize(self):
        """
        Returns the total amount of data contained in the parcel.
        """
        return self._data_size

    def marshall(self):
        """
        Returns the raw bytes of the parcel.
        """
        return self._buffer
    
    def setDataCapacity(self, size):
        """
        Change the capacity (current available space) of the parcel.
        """
        datasize = self.dataSize()
        if size > datasize:
            delta = size - datasize
            self._buffer = self._buffer[:datasize] + array.array('c', '\0' * delta)
        pass

    def setDataPosition(self, pos):
        """
        Move the current read/write position in the parcel.
        """
        self._pointer = pos

    def setDataSize(self, size):
        """
        Change the amount of data in the parcel.
        """
        self._data_amount = size

    def unmarshall(self, data, offset, length):
        """
        Set the bytes in data to be the raw bytes of this Parcel.
        """
        pass
    
    def _writeData(self, dataType, value):
        s = self.primitives[dataType]
        bytesToWrite = s.size
        dataSize = self.dataSize()
        if self.dataCapacity() - dataSize < bytesToWrite:
            size = self.dataCapacity() + self.block
            self.setDataCapacity(size)
        pos = self.dataPosition()
        s.pack_into(self._buffer, pos, value)
        self.setDataPosition(pos + bytesToWrite)
        self.setDataSize(dataSize + bytesToWrite)
        
    def _readData(self, dataType):
        s = self.primitives[dataType]
        bytesToRead = s.size
        if self.dataAvail() < bytesToRead:
            return None
        pos = self.dataPosition()
        self.setDataPosition(pos + bytesToRead)
        return s.unpack_from(self._buffer, pos)
        
        
    
    def readByte(self):
        """
        Read a byte value from the parcel at the current dataPosition().
        """
        return self._readData('byte')
    
    def readDouble(self):
        """
        Read a double precision floating point value from the parcel at the
        current dataPosition().
        """
        return self._readData('double')

    def readFloat(self):
        """
        Read a floating point value from the parcel at the current
        dataPosition().
        """
        return self._readData('float')

    def readInt(self):
        """
        Read an integer value from the parcel at the current dataPosition().
        """
        return self._readData('integer')

    def readLong(self):
        """
        Read a long integer value from the parcel at the current dataPosition().
        """
        return self._readData('long')

    def readString(self):
        """
        Read a string value from the parcel at the current dataPosition().
        """
        return self._readData('string')

    def writeByte(self, val):
        """
        Write a byte value into the parcel at the current dataPosition(),
        growing dataCapacity() if needed.
        """
        self._writeData('byte', val)
        pass

    def writeDouble(self, val):
        """
        Write a double precision floating point value into the parcel at the
        current dataPosition(), growing dataCapacity() if needed.
        """
        self._writeData('double', val)
        pass

    def writeFloat(self, val):
        """
        Write a floating point value into the parcel at the current
        dataPosition(), growing dataCapacity() if needed.
        """
        self._writeData('float', val)
        pass

    def writeInt(self, val):
        """
        Write an integer value into the parcel at the current dataPosition(),
        growing dataCapacity() if needed.
        """
        self._writeData('integer', val)
        pass
    
    def writeLong(self, val):
        """
        Write a long integer value into the parcel at the current dataPosition(),
        growing dataCapacity() if needed.
        """
        self._writeData('long', val)
        pass

    def writeString(self, val):
        """
        Write a string value into the parcel at the current dataPosition(),
        growing dataCapacity() if needed.
        """
        self._writeData('string', val)
        pass
    
    

In [69]:
p = Parcel()

In [70]:
p.writeInt(25)
p.writeFloat(2.5)


In [71]:
p.setDataPosition(0)
print p.readInt()
print p.readFloat()

None
None


In [63]:
p.marshall()

array('c', '\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [67]:
p.dataSize()

0

In [7]:
b = array.array('c', '\0' * s.size)
a.extend(b)
a

array('c', '\x01\x00\x00\x00ab\x00\x00\xcd\xcc,@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [8]:
len(a)

24

In [10]:
s.size

12

In [11]:
s.pack(*values)

'\x01\x00\x00\x00ab\x00\x00\xcd\xcc,@'

In [25]:
        primitives = [('byte', 'h'),
                     ('double', 'd'),
                     ('float', 'f'),
                     ('integer', 'i'),
                     ('long', 'q'),
                     ('string', 'p')
                     ]
        primitives = dict((x[0], struct.Struct(x[1])) for x in primitives)


In [26]:
for key, value in primitives.items():
    print key, value.size

string 1
double 8
float 4
long 8
integer 4
byte 2


In [44]:
buffer = array.array('c', 30*'\0')
pos = 0

In [48]:
dataType = 'integer'
value = 325
s = primitives[dataType]
bytesToAdd = s.size
bytesToAdd

4

In [49]:
s.pack_into(buffer, pos, value)
pos += bytesToAdd

In [50]:
buffer

array('c', '\x00\x00P@E\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [51]:
pos

8